In [9]:
import os  # 운영체제 관련 기능(환경변수 접근 등)을 사용하기 위한 모듈
import requests  # HTTP 요청을 보내기 위한 모듈
from typing import Type  # 타입 힌트를 위한 모듈
from langchain.chat_models import ChatOpenAI  # LangChain의 OpenAI 채팅 모델 래퍼
from langchain.tools import BaseTool  # 도구(tool) 기반 클래스를 제공
from pydantic import BaseModel, Field  # 데이터 검증 및 스키마 정의를 위한 모듈
from langchain.agents import initialize_agent, AgentType  # 에이전트 초기화 및 타입 설정을 위한 모듈
from langchain.utilities import DuckDuckGoSearchAPIWrapper  # DuckDuckGo 검색 API를 활용하기 위한 유틸리티

# OpenAI의 채팅 모델을 초기화합니다.
llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")

# 환경변수에서 Alpha Vantage API 키를 가져옵니다.
alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


# 주식 심볼 검색 도구에 사용할 인자(schema)를 정의합니다.
class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="검색할 쿼리. 예: Apple Company의 주식 시장 심볼"
    )


# 회사의 주식 심볼을 찾기 위한 도구 클래스입니다.
class StockMarketSymbolSearchTool(BaseTool):
    name: str = "StockMarketSymbolSearchTool"  # 도구 이름
    description: str = """
    회사의 주식 시장 심볼을 찾기 위해 사용하는 도구입니다.
    쿼리를 인자로 받아 심볼 정보를 검색합니다.
    """
    # 위에서 정의한 인자 스키마를 사용합니다.
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    # 도구의 실행 메서드. DuckDuckGoSearchAPIWrapper를 사용하여 검색합니다.
    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


# 회사 관련 정보를 조회하기 위한 인자(schema)를 정의합니다.
class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(
        description="회사의 주식 심볼. 예: AAPL, TSLA",
    )


# 회사 개요(재무 개요) 정보를 가져오는 도구 클래스입니다.
class CompanyOverviewTool(BaseTool):
    name:str = "CompanyOverview"  # 도구 이름
    description: str = """
    회사의 재무 개요를 조회하는 도구입니다.
    주식 심볼을 입력하면 회사의 개요 정보를 반환합니다.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    # Alpha Vantage API의 OVERVIEW 함수를 호출하여 회사 개요 정보를 가져옵니다.
    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


# 회사의 손익계산서 정보를 가져오는 도구 클래스입니다.
class CompanyIncomeStatementTool(BaseTool):
    name:str = "CompanyIncomeStatement"  # 도구 이름
    description:str = """
    회사의 손익계산서를 조회하는 도구입니다.
    주식 심볼을 입력하면 손익계산서 정보를 반환합니다.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    # Alpha Vantage API의 INCOME_STATEMENT 함수를 호출하여 손익계산서 정보를 가져옵니다.
    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        # annualReports 섹션만 반환합니다.
        return r.json()["annualReports"]


# 회사의 주식 주간 성과 데이터를 가져오는 도구 클래스입니다.
class CompanyStockPerformanceTool(BaseTool):
    name:str = "CompanyStockPerformance"  # 도구 이름
    description:str = """
    회사의 주식 주간 성과 데이터를 조회하는 도구입니다.
    주식 심볼을 입력하면 주간 성과 정보를 반환합니다.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    # Alpha Vantage API의 TIME_SERIES_WEEKLY 함수를 호출하여 주간 주가 데이터를 가져옵니다.
    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


# 에이전트를 초기화합니다.
# 여기서는 위에서 정의한 네 가지 도구들을 등록하고, OpenAI 함수 호출 방식을 사용하는 에이전트를 생성합니다.
agent = initialize_agent(
    llm=llm,
    verbose=True,  # 실행 시 로그를 출력합니다.
    agent=AgentType.OPENAI_FUNCTIONS,  # OpenAI 함수 호출을 사용하는 에이전트 타입입니다.
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)

# 분석할 프롬프트를 정의합니다.
prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

# 에이전트를 실행하여 프롬프트에 대한 결과를 얻습니다.
agent.invoke(prompt)




> Entering new AgentExecutor chain...


KeyboardInterrupt: 